In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [7]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -3.062457e-06                  NaN           NaN
0.000312      NaN           NaN                  1.0  4.756061e-06
0.000750      2.0 -3.074693e-06                  2.0  1.629297e-05
0.001052      3.0 -3.096656e-06                  3.0 -1.580622e-06
0.001476      4.0 -3.117577e-06                  4.0 -1.852278e-06
0.002070      5.0 -3.136871e-06                  5.0 -2.002918e-06
0.002904      6.0 -3.154710e-06                  6.0 -2.132184e-06
0.004074      7.0 -3.170994e-06                  7.0 -2.092428e-06
0.005714      8.0 -3.184315e-06                  8.0 -2.228148e-06
0.008015      9.0 -3.195763e-06                  9.0 -2.211572e-06
0.011243     10.0 -3.205389e-06                 10.0 -2.267916e-06
0.015771     11.0 -3.212033e-06                 11.0 -2.305447e-06
0.022122     12.0 -3.216225e-06                 12.0 -2.318667e-06
0.031031     13.0 -3.217972e-06                 13.0 -2.071963e-06
0.043528     14.0 -3.215579e-06                 14.0  1.263531e-07
0.061057     15.0 -3.164832e-06                 15.0  5.175423e-07
0.085645     16.0 -2.811961e-06                 16.0 -1.067390e-06
0.120136     17.0 -1.774018e-06                 17.0 -1.529110e-06
0.168516     18.0  3.174696e-07                 18.0 -9.819357e-07
0.236378     19.0  4.012051e-06                 19.0  2.356098e-07
0.331549     20.0  1.008080e-05                 20.0  2.395362e-06
0.465100     21.0  1.958012e-05                 21.0  5.761290e-06
0.652400     22.0  3.229261e-05                 22.0  1.011045e-05
0.915100     23.0  3.763563e-05                 23.0  1.257239e-05
1.283650     24.0  3.244320e-05                 24.0  1.098043e-05
1.800600     25.0  2.441988e-05                 25.0  7.833521e-06
2.525700     26.0  1.578892e-05                 26.0  4.472191e-06
3.542800     27.0  9.067818e-06                 27.0  1.829006e-06
4.969550     28.0  4.365580e-06                 28.0 -7.275031e-09
6.970850     29.0  1.158907e-06                 29.0 -1.233977e-06
9.778100     30.0 -9.564875e-07                 30.0 -2.016138e-06
13.715850    31.0 -2.266144e-06                 31.0 -2.476692e-06
19.239350    32.0 -3.003623e-06                 32.0 -2.710565e-06
26.987250    33.0 -3.342380e-06                 33.0 -2.776050e-06
37.855300    34.0 -3.421479e-06                 34.0 -2.706791e-06
53.100050    35.0 -3.231183e-06                 35.0 -2.577504e-06
73.887500    36.0 -2.770823e-06                 36.0 -2.360606e-06
97.662500    37.0 -2.478024e-06                 37.0 -2.215336e-06
121.437500   38.0 -2.769983e-06                 38.0 -2.490245e-06
145.212500   39.0 -3.486453e-06                 39.0 -3.079855e-06
168.987500   40.0 -4.530767e-06                 40.0 -3.976397e-06
192.762500   41.0 -7.396834e-06                 41.0 -6.673456e-06
216.537500   42.0 -1.759436e-05                 42.0 -1.692388e-05
240.312500   43.0 -3.432805e-05                 43.0 -3.564118e-05
264.087500   44.0 -5.578136e-05                 44.0 -6.387780e-05
287.862500   45.0 -7.422477e-05                 45.0 -9.907408e-05
311.637500   46.0 -7.366136e-05                 46.0 -1.321717e-04
335.412500   47.0 -2.859422e-05                 47.0 -1.435591e-04
359.187500   48.0  9.427437e-05                 48.0 -1.045170e-04
382.962500   49.0  3.155546e-04                 49.0  2.130092e-05
406.737500   50.0  6.666359e-04                 50.0  2.579832e-04
430.512500   51.0  1.181950e-03                 51.0  6.400278e-04
454.287500   52.0  1.897115e-03                 52.0  1.212542e-03
478.062500   53.0  2.849152e-03                 53.0  2.020817e-03
501.837500   54.0  4.069813e-03                 54.0  3.103067e-03
525.612500   55.0  5.603002e-03                 55.0  4.453681e-03
549.387500   56.0  7.479904e-03                 56.0  6.12932

# Fluxes by Layer

CRD                              CLIRAD            \
                         flug          fldg      fnetg       flug      fldg   
pressure     level                                                            
0.000000e+00 1     -33.973919  0.000000e+00 -33.973919        NaN       NaN   
1.000000e-08 1            NaN           NaN        NaN -34.013677  0.000002   
6.244000e-04 2     -33.973919  8.282641e-11 -33.973919 -34.013677  0.000002   
8.759000e-04 3     -33.973919  1.168927e-10 -33.973919 -34.013677  0.000002   
1.228600e-03 4     -33.973919  1.667405e-10 -33.973919 -34.013677  0.000002   
1.723400e-03 5     -33.973919  2.397411e-10 -33.973919 -34.013677  0.000002   
2.417400e-03 6     -33.973919  3.466170e-10 -33.973919 -34.013677  0.000002   
3.390900e-03 7     -33.973919  5.030999e-10 -33.973919 -34.013677  0.000002   
4.756500e-03 8     -33.973919  7.321728e-10 -33.973919 -34.013677  0.000002   
6.672000e-03 9     -33.973919  1.067444e-09 -33.973919 -34.013677  0.000002   
9.358900e-03 10    -33.973919  1.558089e-09 -33.973919 -34.013677  0.000002   
1.312780e-02 11    -33.973919  2.276009e-09 -33.973919 -34.013677  0.000002   
1.841450e-02 12    -33.973919  3.326359e-09 -33.973919 -34.013677  0.000002   
2.583020e-02 13    -33.973919  4.862872e-09 -33.973919 -34.013677  0.000002   
3.623230e-02 14    -33.973919  7.110629e-09 -33.973919 -34.013677  0.000002   
5.082340e-02 15    -33.973919  1.043642e-08 -33.973919 -34.013677  0.000002   
7.129060e-02 16    -33.973919  1.605156e-08 -33.973919 -34.013677  0.000002   
1.000000e-01 17    -33.973919  2.734352e-08 -33.973919 -34.013677  0.000002   
1.402710e-01 18    -33.973919  5.122416e-08 -33.973919 -34.013677  0.000002   
1.967600e-01 19    -33.973919  1.012709e-07 -33.973919 -34.013677  0.000002   
2.759970e-01 20    -33.973919  2.050672e-07 -33.973919 -34.013677  0.000002   
3.871000e-01 21    -33.973919  4.180737e-07 -33.973919 -34.013677  0.000002   
5.431000e-01 22    -33.973919  8.513531e-07 -33.973918 -34.013678  0.000002   
7.617000e-01 23    -33.973919  1.692804e-06 -33.973918 -34.013678  0.000002   
1.068500e+00 24    -33.973919  3.007065e-06 -33.973916 -34.013678  0.000003   
1.498800e+00 25    -33.973919  4.668908e-06 -33.973915 -34.013678  0.000004   
2.102400e+00 26    -33.973919  6.597135e-06 -33.973913 -34.013678  0.000005   
2.949000e+00 27    -33.973920  8.667058e-06 -33.973911 -34.013679  0.000006   
4.136600e+00 28    -33.973921  1.083249e-05 -33.973910 -34.013680  0.000007   
5.802500e+00 29    -33.973922  1.308661e-05 -33.973909 -34.013681  0.000008   
8.139200e+00 30    -33.973924  1.541953e-05 -33.973909 -34.013682  0.000009   
1.141700e+01 31    -33.973927  1.781527e-05 -33.973909 -34.013684  0.000010   
1.601470e+01 32    -33.973931  2.026139e-05 -33.973910 -34.013687  0.000012   
2.246400e+01 33    -33.973935  2.274586e-05 -33.973913 -34.013690  0.000013   
3.151050e+01 34    -33.973942  2.525881e-05 -33.973916 -34.013695  0.000014   
4.420010e+01 35    -33.973949  2.775312e-05 -33.973921 -34.013700  0.000016   
6.200000e+01 36    -33.973958  2.992945e-05 -33.973928 -34.013707  0.000017   
8.577500e+01 37    -33.973968  3.150287e-05 -33.973936 -34.013715  0.000018   
1.095500e+02 38    -33.973976  3.259078e-05 -33.973943 -34.013722  0.000019   
1.333250e+02 39    -33.973985  3.401840e-05 -33.973951 -34.013730  0.000020   
1.571000e+02 40    -33.973997  3.664120e-05 -33.973961 -34.013741  0.000022   
1.808750e+02 41    -33.974015  4.147665e-05 -33.973973 -34.013755  0.000026   
2.046500e+02 42    -33.974047  5.254247e-05 -33.973994 -34.013782  0.000033   
2.284250e+02 43    -33.974133  8.960008e-05 -33.974044 -34.013855  0.000059   
2.522000e+02 44    -33.974334  1.935144e-04 -33.974141 -34.014028  0.000132   
2.759750e+02 45    -33.974745  4.469162e-04 -33.974298 -34.014387  0.000310   
2.997500e+02 46    -33.975505  9.986421e-04 -33.974507 -34.015057  0.000702   
3.235250e+02 47    -33.976806  2.091519e-03 -33.974714 -34.016214  0.001486 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')